# ChemCluster Project Report

**CH-200 – Practical Programming in Chemistry**  
**Group:** Elisa Rubbia, Romain Guichonnet, Flavia Zabala Perez  
**Date:** May 2025

## Welcome to ChemCluster!

ChemCluster is a streamlined and interactive platform designed to facilitate the analysis and visualization of chemical datasets using molecular clustering techniques. It leverages the power of **RDKit** for cheminformatics and **Streamlit** for web interface deployment, offering an intuitive solution for researchers and students alike.

Two main modes of operation are supported by the platform:

- **Dataset Mode**: This mode allows users to upload datasets (in `.sdf`, `.mol` or `.csv` format) containing multiple molecular structures.  The molecules are processed to compute their key physicochemical descriptors, then clustered based on these properties. The resulting clusters are visualized interactively, and users can explore individual structures along with their 2D/3D representations and calculated properties.
- **Single Molecule Mode**: In this workflow, ChemCluster generates a set of conformers for a single input molecule. These conformers are clustered based on structural similarity, and representative structures are visualized in  3D, enabling users to analyze conformational diversity effectively.

## 1. Setup and Initialization

To begin using ChemCluster, you have two options: installation via PyPI or running the application locally from source.

### 1.1 Installation from PyPI
The simplest way to install ChemCluster is through the Python Package Index. In a terminal or command prompt, run:

```bash 
pip install chemcluster 
``` 

Once installed, you can launch the application by executing:

```bash
chemcluster
```
This command will open the ChemCluster interface directly in your default web browser.

### 1.2 Running Locally from Source (for Development or Contribution)
If you prefer to contribute to the project or wish to run it locally in a development environment, follow these steps:
1. Clone the repository from GitHub:
```bash
git clone https://github.com/erubbia/ChemCluster
```
2. Navigate into the project directory:
```bash
cd ChemCluster
```
3. Create a conda environment based on the project's environment file:
```bash
conda env create -f environment.yml
```
4. Activate the newly created environment:
```bash
conda activate chemcluster-env
```
5. Finally, install the project in editable mode:
```bash
pip install -e .
```
After this setup, you can launch ChemCluster the same way by running `chemcluster` in the terminal.

## 2. Dataset Mode – Analysis of Molecular Libraries

The dataset mode enables users to upload and analyze molecular datasets containing multiple compounds. This mode is suitable for exploring the diversity of a chemical library, identifying representative clusters, and filtering molecules based on specific chemical properties.

The clustering workflow is based on molecular descriptors derived from circular fingerprints, and involves dimensionality reduction and unsupervised clustering. Molecules within clusters can be interactively explored and exported for further analysis.

### 2.1 Data Input and Preprocessing
Users can upload molecular datasets in `.csv`, `.sdf`, or `.mol` formats. For `.csv` files, the application searches for a column named `SMILES` or similar, and converts each string into an RDKit `Mol` object. These are then passed through the `clean_smiles_list()` function, which removes invalid or unreadable entries to ensure robust downstream processing.

```python
from chemcluster import clean_smiles_list
mols, smiles_list = clean_smiles_list(smiles_column)
```

### 2.2 Descriptor Calculation and Fingerprint Generation
Each valid molecule is encoded into a binary fingerprint vector using RDKit's implementation of Morgan fingerprints. This is achieved via the `get_fingerprint()` function, which returns a 2048-bit representation capturing the molecular environment of atoms.

The similarity between all pairs of molecules is then calculated using the Tanimoto coefficient, a widely used metric for binary vector comparison:

In [ ]:

from chemcluster import get_fingerprint
fps = [get_fingerprint(mol) for mol in mols]
from rdkit import DataStructs
similarity = DataStructs.TanimotoSimilarity(fps[i], fps[j])

These similarities are converted into a distance matrix by computing `1 - similarity`, which serves as input for PCA.

### 2.3 Dimensionality Reduction
To facilitate visualization and clustering, the high-dimensional distance matrix is projected into a 2D space using Principal Component Analysis (PCA). PCA reduces the complexity of the data while retaining the directions of maximum variance.

The transformation is performed using `sklearn.decomposition.PCA`, producing 2D coordinates that reflect the relative positions of molecules in descriptor space:
```python
from sklearn.decomposition import PCA
coords = PCA(n_components=2).fit_transform(dist_matrix)
```

### 2.4 Clustering with KMeans
Molecules are grouped into clusters using the KMeans algorithm from `sklearn.cluster`. The optimal number of clusters `k` is determined by evaluating the silhouette score over a range of possible values (typically k = 2 to 10). The silhouette score quantifies both the cohesion within clusters and the separation between them.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
best_k, best_score = 2, -1
for k in range(2, 11):
    model = KMeans(n_clusters=k).fit(coords)
    score = silhouette_score(coords, model.labels_)
    if score > best_score:
        best_k, best_score = k, score
labels = KMeans(n_clusters=best_k).fit_predict(coords)

### 2.5 Visualization and Molecule Inspection
The clustering results are displayed as a 2D scatter plot using Plotly, where each point corresponds to a molecule in PCA space. The `plotly_events()` function captures user clicks on individual points, triggering the display of the molecule’s structure, 3D viewer, and properties.

In [ ]:
from streamlit_plotly_events import plotly_events
selected_points = plotly_events(fig, click_event=True)

This interactivity enables quick identification and inspection of interesting chemical structures.

### 2.6 Cluster Filtering by Properties
For each cluster, ChemCluster calculates the average values of selected molecular descriptors using the `calculate_properties()` function. This information is used to filter clusters that match user-defined criteria, such as high LogP or low TPSA.

In [ ]:
cluster_props_summary = {}
for c in set(labels):
    idxs = [i for i, lbl in enumerate(labels) if lbl == c]
    props = [calculate_properties(mols[i]) for i in idxs]
    df = pd.DataFrame(props).select_dtypes(include='number')
    cluster_props_summary[c] = df.mean()

### 2.7 Export Functionality
The filtered clusters and their associated molecular data can be exported in `.csv` format for documentation or further analysis. This is handled using Pandas’ `to_csv()` method and Streamlit’s `download_button()`:

In [ ]:
csv = cluster_df.to_csv(index=False).encode('utf-8')
st.download_button("Download Cluster Molecules", data=csv,
                   file_name="cluster_data.csv", mime="text/csv")

## 3. Single Molecule Mode – Conformer Generation and Visualization

In Single Molecule Mode, ChemCluster allows users to enter or draw a molecule, generate 3D conformers, perform clustering based on conformational similarity, and visualize representative conformations.

This mode is particularly useful for analyzing the conformational landscape of flexible molecules or inspecting representative shapes for downstream modeling.

### 3.1 Molecular Input
The user can enter a molecule via a SMILES string or draw it interactively using an embedded chemical editor. The SMILES string is parsed with RDKit to generate a `Mol` object.

In [ ]:
from rdkit import Chem
mol = Chem.MolFromSmiles("CCO")

### 3.2 Conformer Generation
RDKit's ETKDG algorithm is used to generate multiple conformers for the input molecule. Explicit hydrogen atoms are added prior to embedding to improve 3D accuracy. The user can specify the number of conformers to be generated.


In [ ]:
from rdkit.Chem import AllChem
mol = Chem.AddHs(mol)
cids = AllChem.EmbedMultipleConfs(mol, numConfs=50, randomSeed=42)
AllChem.UFFOptimizeMoleculeConfs(mol)

### 3.3 Conformer Clustering
The conformers are clustered based on their pairwise RMSD values using the Butina clustering algorithm. This allows identification of conformational families. The conformer with the lowest average RMSD within each cluster is selected as the centroid.

In [ ]:
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolAlign
dists = []
for i in range(len(cids)):
    for j in range(i):
        rms = rdMolAlign.GetBestRMS(mol, mol, i, j)
        dists.append(rms)
clusters = Butina.ClusterData(dists, len(cids), 1.5, isDistData=True)

### 3.4 Visualization of Centroids
Users can select centroid conformers to visualize them in 3D using py3Dmol. This allows comparing conformational representatives interactively.

In [ ]:
import py3Dmol
viewer = py3Dmol.view(width=400, height=400)
mb = Chem.MolToMolBlock(mol, confId=centroid_id)
viewer.addModel(mb, 'mol')
viewer.setStyle({'stick': {}})
viewer.zoomTo()

### 3.5 Property Calculation for Centroids
For each centroid conformer, the `calculate_properties()` function is used to compute relevant physicochemical descriptors. These include molecular weight, LogP, number of hydrogen bond donors/acceptors, TPSA, rotatable bonds, etc.

In [ ]:
from chemcluster import calculate_properties
props = calculate_properties(mol, mol_name="example")

The dataset mode enables users to upload and analyze molecular datasets containing multiple compounds. This mode is suitable for exploring the diversity of a chemical library, identifying representative clusters, and filtering molecules based on specific chemical properties.

The clustering workflow is based on molecular descriptors derived from circular fingerprints, and involves dimensionality reduction and unsupervised clustering. Molecules within clusters can be interactively explored and exported for further analysis.

## 4. Results and Observations

To demonstrate the functionality of ChemCluster, we considered testing the application with a small molecule input. One possible example is the flavone molecule, provided as an `.sdf` file (`Flavone.sdf`).

In single molecule mode, ChemCluster successfully processed the molecule, added explicit hydrogens, and generated a set of conformers using the ETKDG algorithm. The generated conformers were clustered based on RMSD distances using the Butina algorithm. The application identified a small number of representative conformers (cluster centroids), which were then visualized in both 2D and 3D.

The flavone molecule’s physicochemical properties — including molecular weight, LogP, TPSA, and hydrogen bonding features — were computed and displayed within the application interface. All values were consistent with expectations for a moderately hydrophobic, aromatic compound.

The export feature allowed downloading all conformer and property data in a structured `.csv` format. This facilitates further analysis or integration with external cheminformatics workflows.

*Note: A final version of this section will include updated values and screenshots based on the exact molecule used during the oral demonstration.*


## 5. Testing and Validation

To ensure reliability and maintainability, ChemCluster includes a suite of unit tests covering all core functionalities. These tests are located in the `tests/` directory and can be executed using either `pytest` or `tox`, ensuring compatibility across Python versions.

### 5.1 Coverage and Scope
The following components are covered by unit tests:
- `clean_smiles_list`: checks for SMILES parsing and molecule validity
- `get_fingerprint`: verifies correct generation of Morgan fingerprints
- `calculate_properties`: ensures property values match expected reference
- `mol_to_base64_img`: validates base64 image string formatting
- `show_3d_molecule`: checks py3Dmol viewer object creation

### 5.2 Example Test Snippet

In [ ]:
def test_calculate_properties():
    mol = Chem.MolFromSmiles("CCO")  # ethanol
    result = calculate_properties(mol, mol_name="ethanol")
    assert isinstance(result, dict)
    assert abs(result["Molecular Weight"] - 46.07) < 0.1

## 6. Challenges Faced

While the development of ChemCluster was overall successful and rewarding, several technical and practical challenges emerged during the implementation and testing phases:

- **Performance with large datasets or high conformer counts:**  
  When processing large molecule sets or generating a high number of conformers, the application experienced significant slowdowns during 3D generation and clustering. This was particularly evident with long optimization times and memory consumption.

- **3D visualization issues with certain SMILES:**  
  Although py3Dmol generally worked well, some SMILES strings led to unexpected 3D representations or viewer artifacts. This may be linked to problematic stereochemistry or non-standard atom arrangements in the input data.

- **Cross-platform compatibility limitations:**  
  The app's dependencies (e.g. `py3Dmol`, `streamlit_plotly_events`) occasionally behaved differently between Windows and MacOS environments. Some packages required manual installation or specific version pinning depending on the OS.

These limitations were addressed when possible and are documented for future improvements.



## 7. Conclusion and Outlook

ChemCluster has proven to be a functional and user-friendly cheminformatics platform for exploring molecular structures and clusters through RDKit and Streamlit. The application allows both single-molecule and dataset-based analyses, including conformer generation, clustering, property calculation, 2D/3D visualization, and interactive selection.

The project successfully integrates key aspects of molecular analysis into a cohesive, visual, and reproducible workflow. It has been used to process real molecular inputs such as flavone, and demonstrates clear separation of molecular clusters, meaningful descriptor distributions, and smooth user interaction.

While the main functionalities are complete and operational, additional features were considered during the development process. These include more advanced modeling or solution-phase interaction analysis, which were postponed in favor of keeping the application focused, functional, and robust.

### Future Perspective

A promising extension could be the visualization of hydrogen bonding interactions between different centroid conformers of a given molecule in solution. This would help investigate the possible over- or underestimation of transition states (TS) depending on conformer environments. Although this idea was not implemented in the final version, it remains a valuable direction for future work.
